In [1]:
import matplotlib.pyplot as plt
import numpy as np
import graphviz

import gtsam
import gtsam.utils.plot as gp

# We can use shorthand symbols for variable keys
# X(i) represents the i-th pose variable
# L(j) represents the j-th landmark variable
from gtsam.symbol_shorthand import L, X

In [2]:
# Create noise models with specified standard deviations (sigmas).
# gtsam.noiseModel.Diagonal.Sigmas takes a numpy array of standard deviations.

# Prior noise on the first pose (x, y, theta) - sigmas = [0.3m, 0.3m, 0.1rad]
PRIOR_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
# Odometry noise (dx, dy, dtheta) - sigmas = [0.2m, 0.2m, 0.1rad]
ODOMETRY_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
# Measurement noise (bearing, range) - sigmas = [0.1rad, 0.2m]
MEASUREMENT_NOISE = gtsam.noiseModel.Diagonal.Sigmas(np.array([0.1, 0.2]))

In [3]:
# Create an empty nonlinear factor graph
graph = gtsam.NonlinearFactorGraph()

In [4]:
# Add a prior on pose X(1) at the origin.
# A prior factor consists of a mean (gtsam.Pose2) and a noise model.
graph.add(gtsam.PriorFactorPose2(X(1), gtsam.Pose2(0.0, 0.0, 0.0), PRIOR_NOISE))

In [5]:
# Add odometry factors between X(1),X(2) and X(2),X(3), respectively.
# The measurement is the relative motion: Pose2(dx, dy, dtheta).

# Between X(1) and X(2): Move forward 2m
graph.add(gtsam.BetweenFactorPose2(X(1), X(2), gtsam.Pose2(2.0, 0.0, 0.0), ODOMETRY_NOISE))
# Between X(2) and X(3): Move forward 2m
graph.add(gtsam.BetweenFactorPose2(X(2), X(3), gtsam.Pose2(2.0, 0.0, 0.0), ODOMETRY_NOISE))

In [6]:

# From X(1) to L(1)
graph.add(gtsam.BearingRangeFactor2D(X(1), L(1), gtsam.Rot2.fromDegrees(45), np.sqrt(4.0+4.0), MEASUREMENT_NOISE))
# From X(2) to L(1)
graph.add(gtsam.BearingRangeFactor2D(X(2), L(1), gtsam.Rot2.fromDegrees(90), 2.0, MEASUREMENT_NOISE))
# From X(3) to L(2)
graph.add(gtsam.BearingRangeFactor2D(X(3), L(2), gtsam.Rot2.fromDegrees(90), 2.0, MEASUREMENT_NOISE))

In [9]:
# Create (deliberately inaccurate) initial estimate.
# gtsam.Values is a container mapping variable keys to their estimated values.
initial_estimate = gtsam.Values()

# Insert initial guesses for poses (Pose2: x, y, theta)
initial_estimate.insert(X(1), gtsam.Pose2(-0.25, 0.20, 0.15))
initial_estimate.insert(X(2), gtsam.Pose2(2.30, 0.10, -0.20))
initial_estimate.insert(X(3), gtsam.Pose2(4.10, 0.10, 0.10))

# Insert initial guesses for landmarks (Point2: x, y)
initial_estimate.insert(L(1), gtsam.Point2(1.80, 2.10))
initial_estimate.insert(L(2), gtsam.Point2(4.10, 1.80))

# Print the initial estimate
print("Initial Estimate:\n{}".format(initial_estimate))

Initial Estimate:
Values with 5 values:
Value l1: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	1.8;
	2.1
]

Value l2: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	4.1;
	1.8
]

Value x1: (gtsam::Pose2)
(-0.25, 0.2, 0.15)

Value x2: (gtsam::Pose2)
(2.3, 0.1, -0.2)

Value x3: (gtsam::Pose2)
(4.1, 0.1, 0.1)




In [18]:
# Write initial (unoptimized) values to planar_unoptimized.txt
with open('data/planar_unoptimized.txt', 'w') as f:
    for i in range(1, 4):
        pose = initial_estimate.atPose2(X(i))
        f.write(f"{pose.x()} {pose.y()} {pose.theta()}\n")
print("Unoptimized values written to data/planar_unoptimized.txt")

Unoptimized values written to data/planar_unoptimized.txt


In [12]:
# Optimize using Levenberg-Marquardt optimization.
# The optimizer accepts optional parameters, but we'll use the defaults here.
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)

# Perform the optimization
result = optimizer.optimize()

# Print the final optimized result
# This gtsam.Values object contains the most likely estimates for all variables.
print("\nFinal Result:\n{}".format(result))


Final Result:
Values with 5 values:
Value l1: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	2;
	2
]

Value l2: (Eigen::Matrix<double, -1, 1, 0, -1, 1>)
[
	4;
	2
]

Value x1: (gtsam::Pose2)
(-7.45949e-16, -5.78962e-16, -2.45047e-16)

Value x2: (gtsam::Pose2)
(2, -1.06329e-14, -1.33676e-15)

Value x3: (gtsam::Pose2)
(4, -3.71e-14, -1.30673e-15)




In [ ]:
# Write optimized values to planar_optimized.txt
with open('data/planar_optimized.txt', 'w') as f:
    for i in range(1, 4):
        pose = result.atPose2(X(i))
        f.write(f"{pose.x()} {pose.y()} {pose.theta()}\n")
print("Optimized values written to data/planar_optimized.txt")

Optimized values written to data/planar_optimized.txt
